In [2]:
!pip install -q kaggle
!pip install -q simpletransformers
!mkdir .kaggle

import json
token = {"username":"......","key":'......."}
with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

!cp .kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v{/content}
!chmod 600 /root/.kaggle/kaggle.json
print('Скачиваю датасеты')
!kaggle competitions download -c sf-dl-movie-genre-classification -p /content
!ls

mkdir: cannot create directory ‘.kaggle’: File exists
- path is now set to: {/content}
Скачиваю датасеты
  0% 0.00/718k [00:00<?, ?B/s]
100% 718k/718k [00:00<00:00, 48.8MB/s]
 65% 9.00M/13.8M [00:00<00:00, 24.7MB/s]
100% 13.8M/13.8M [00:00<00:00, 34.8MB/s]
 36% 5.00M/13.9M [00:00<00:00, 17.6MB/s]
100% 13.9M/13.9M [00:00<00:00, 35.2MB/s]
 37% 5.00M/13.7M [00:00<00:00, 13.2MB/s]
100% 13.7M/13.7M [00:00<00:00, 30.8MB/s]
 65% 9.00M/13.8M [00:00<00:00, 24.3MB/s]
100% 13.8M/13.8M [00:00<00:00, 34.8MB/s]
sample_data	       test.csv.zip	  train.csv.zip
sample-submission.csv  test_data.txt.zip  train_data.txt.zip


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import random
import numpy as np 
import pandas as pd 

from simpletransformers.classification import ClassificationModel

In [0]:
train = pd.read_csv('train.csv.zip')
test = pd.read_csv('test.csv.zip')
submission = pd.read_csv('sample-submission.csv')

train['new_str'] = train['name'] + train['text']
train = train[['new_str', 'genre']]

test['text'] = test['name'] + test['text']
test.drop(['name'], axis = 1, inplace = True)

classes =  list(set(train.genre))
b = {}
a = 0
for i in classes:
  b[i] = a
  a += 1
train['genre'] = train['genre'].map(b)

In [6]:
model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=27, args={"num_train_epochs": 2,
                                                                              "fp16": False, 
                                                                              'overwrite_output_dir': True})

In [7]:
model.train_model(train)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:251: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 3.476690

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


Running loss: 3.022914

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


Running loss: 1.386696

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running loss: 0.922809


Running loss: 0.185725



In [8]:
pred, raw_outputs = model.predict(test['text'])
p = {v:k for k, v in b.items()}
submission['genre'] = pred
submission['genre'] = submission['genre'].map(p)

In [0]:
submission.to_csv('/content/drive/My Drive/text_class_submission_11.csv', index=False)
predict_proba = pd.DataFrame(raw_outputs, columns=classes)
predict_proba.to_csv('/content/drive/My Drive/pred_proba_xlnet.csv', index=False)

In [11]:
submission

,id,genre
0,1,thriller
1,2,comedy
2,3,documentary
3,4,drama
4,5,drama
...,...,...
54195,54196,drama
54196,54197,western
54197,54198,comedy
54198,54199,drama


In [12]:
classes

['documentary',
 'biography',
 'adventure',
 'family',
 'talk-show',
 'game-show',
 'crime',
 'musical',
 'fantasy',
 'war',
 'animation',
 'sci-fi',
 'history',
 'news',
 'horror',
 'romance',
 'music',
 'drama',
 'mystery',
 'comedy',
 'adult',
 'sport',
 'action',
 'reality-tv',
 'thriller',
 'short',
 'western']

In [13]:
p

{0: 'documentary',
 1: 'biography',
 2: 'adventure',
 3: 'family',
 4: 'talk-show',
 5: 'game-show',
 6: 'crime',
 7: 'musical',
 8: 'fantasy',
 9: 'war',
 10: 'animation',
 11: 'sci-fi',
 12: 'history',
 13: 'news',
 14: 'horror',
 15: 'romance',
 16: 'music',
 17: 'drama',
 18: 'mystery',
 19: 'comedy',
 20: 'adult',
 21: 'sport',
 22: 'action',
 23: 'reality-tv',
 24: 'thriller',
 25: 'short',
 26: 'western'}